In [5]:
from __future__ import print_function

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import torch



import os
import sys
import glob
import h5py
import numpy as np
import math


import torch
from torch import nn
from torch.autograd import Variable
from torchvision import transforms
from torchvision.utils import save_image
from torch.utils.data import Dataset , DataLoader
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.model_zoo as model_zoo
from tqdm import tqdm
import tensorboard
import tensorboardX
from torch.utils.tensorboard import SummaryWriter


from log import Logger
from data import trainlabelDataset, testlabelDataset
from util import r2, mse, rmse, mae, pp_mse, pp_rmse, pp_mae
from model import  autoencoder_999, autoencoder_333
from sklearn.decomposition import PCA
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


def showres(path):
    img_data_tensor=torch.load(path)
    img_data= img_data_tensor[4].numpy()
    img_data.resize(64,64)
    plt.imshow(img_data)
    return img_data

def to_img(x):   # image size 56 * 56 
    x = x.view(x.size(0), 1, 64, 64)
    return x


test_dataset = testlabelDataset()
test_dataloader= DataLoader(dataset=test_dataset, batch_size=3000,shuffle=False,drop_last=True)
criterion_none = nn.L1Loss(reduction='none')


In [6]:
num=840


#batch_size = 64
#learning_rate = 1e-4
#weight=5e4

model = autoencoder_333().cuda()   ###### AE model 
model.load_state_dict(torch.load('gal_img1001/exp472_{}.pth'.format(num)))    
model.eval()

for data in test_dataloader:
    test_img,test_label= [x.type(torch.float32).cuda() for x in data]
    test_img = test_img.view(test_img.size(0), 1,64,64)
    test_output,test_z = model(test_img)
    test_z=test_z.view(test_z.size(0),14*14)
    
 #   test_loss= (( criterion_none(test_output, test_img)/(test_img.sum(dim=3).sum(dim=2).sum(dim=1))).mean())  + (criterion_none(test_z[:,:7], test_label)  /weight).mean() 
        
#    test_loss_recon= (( criterion_none(test_output, test_img)/(test_img.sum(dim=3).sum(dim=2).sum(dim=1))).mean()) 
 #   test_loss_latent= (criterion_none(test_z[:,:7], test_label)  /weight).mean() 
        
 #   test_MSE_loss = nn.MSELoss()(test_output, test_img)
  

    



res_in =  test_img.data.cpu().numpy()   
res_out=test_output.data.cpu().numpy()



RuntimeError: CUDA out of memory. Tried to allocate 48.00 MiB (GPU 0; 15.90 GiB total capacity; 10.17 GiB already allocated; 21.88 MiB free; 5.11 MiB cached)

In [1]:
test_img.shape

NameError: name 'test_img' is not defined

In [ ]:
img_in=res_in[40][0]
img_out=res_out[40][0]

plt.subplot(1, 3, 1)
plt.imshow(img_in)
plt.subplot(1, 3, 2)  
plt.imshow(img_out) 
plt.subplot(1, 3, 3) 
plt.imshow(img_in-img_out) 
plt.colorbar()

In [ ]:
plt.subplot(1, 3, 1)
a1=showres('gal_img1001/exp472_test_x_840.pt')
plt.subplot(1, 3, 2)
a2=showres('gal_img1001/exp472_test_x_hat_840.pt') 
plt.subplot(1, 3, 3) 
plt.imshow(a1-a2)
plt.colorbar()

In [ ]:
plt.subplot(1, 3, 1)
a1=showres('gal_img1001/exp505_test_x_10.pt')
plt.subplot(1, 3, 2)
a2=showres('gal_img1001/exp505_test_x_hat_10.pt') 
plt.subplot(1, 3, 3) 
plt.imshow(a1-a2)
plt.colorbar()

In [ ]:
plt.subplot(1, 3, 1)
a1=showres('gal_img1001/exp462_test_x_1670.pt')
plt.subplot(1, 3, 2)
a2=showres('gal_img1001/exp462_test_x_hat_1670.pt') 
plt.subplot(1, 3, 3) 
plt.imshow(a1-a2)
plt.colorbar()

In [ ]:
plt.subplot(1, 3, 1)
a1=showres('gal_img1001/exp441_test_x_1200.pt')
plt.subplot(1, 3, 2)
a2=showres('gal_img1001/exp441_test_x_hat_1200.pt') 
plt.subplot(1, 3, 3) 
plt.imshow(a1-a2)
plt.colorbar()

In [ ]:
plt.subplot(1, 3, 1)
a1=showres('gal_img1001/exp462_test_x_800.pt')
plt.subplot(1, 3, 2)
a2=showres('gal_img1001/exp462_test_x_hat_800.pt') 
plt.subplot(1, 3, 3) 
plt.imshow(a1-a2)
plt.colorbar()

In [ ]:
plt.subplot(1, 3, 1)
a1=showres('gal_img1001/exp414_test_x_2000.pt')
plt.subplot(1, 3, 2)
a2=showres('gal_img1001/exp414_test_x_hat_2000.pt') 
plt.subplot(1, 3, 3) 
plt.imshow(a1-a2)
plt.colorbar()

In [ ]:
plt.subplot(1, 3, 1)
a1=showres('gal_img1001/exp416_test_x_900.pt')
plt.subplot(1, 3, 2)
a2=showres('gal_img1001/exp416_test_x_hat_900.pt') 
plt.subplot(1, 3, 3) 
plt.imshow(a1-a2)
plt.colorbar()

In [ ]:
plt.subplot(1, 3, 1)
a1=showres('gal_img1001/exp413_test_x_4900.pt')
plt.subplot(1, 3, 2)
a2=showres('gal_img1001/exp413_test_x_hat_4900.pt') 
plt.subplot(1, 3, 3) 
plt.imshow(a1-a2)
plt.colorbar()

In [ ]:
plt.subplot(1, 3, 1)
a1=showres('gal_img1001/exp423_test_x_100.pt')
plt.subplot(1, 3, 2)
a2=showres('gal_img1001/exp423_test_x_hat_100.pt') 
plt.subplot(1, 3, 3) 
plt.imshow(a1-a2)
plt.colorbar()

In [ ]:
plt.subplot(1, 3, 1)
a1=showres('gal_img1001/exp601_test_x_8800.pt')
plt.subplot(1, 3, 2)
a2=showres('gal_img1001/exp601_test_x_hat_8800.pt') 
plt.subplot(1, 3, 3) 
plt.imshow(a1-a2)
plt.colorbar()

In [ ]:
plt.subplot(1, 3, 1)
a1=showres('gal_img1001/exp451_test_x_3000.pt')
plt.subplot(1, 3, 2)
a2=showres('gal_img1001/exp451_test_x_hat_3000.pt') 
plt.subplot(1, 3, 3) 
plt.imshow(a1-a2)
plt.colorbar()

In [ ]:
plt.subplot(1, 3, 1)
a1=showres('gal_img1001/exp651_test_x_6000.pt')
plt.subplot(1, 3, 2)
a2=showres('gal_img1001/exp651_test_x_hat_6000.pt') 
plt.subplot(1, 3, 3) 
plt.imshow(a1-a2)
plt.colorbar()

In [ ]:
num=100
model = autoencoder_999().cuda()   ###### AE model 
model.load_state_dict(torch.load('gal_img1001/exp202_{}.pth'.format(num)))    
model.eval()


for data in test_dataloader:
    test_img_ = data
    test_img = test_img_.type(torch.float32)
    test_img = test_img.view(test_img.size(0), 1,64,64)
    test_img = test_img.cuda()
    test_img_output, _= model(test_img)

    test_latent = model.encoder(test_img)   #latent 
    
test_img_hat=test_img_output.data.cpu().numpy()



In [ ]:
torch.cuda.empty_cache()

In [ ]:
test_img.shape

In [ ]:
plt.subplot(1, 3, 1)
plt.imshow(test_img_hat[0])
plt.subplot(1, 3, 2)
plt.imshow(test_img_[0])
plt.subplot(1, 3, 3) 
plt.imshow(a1-a2)

In [ ]:
num=1000
model = autoencoder_333().cuda()   ###### AE model 
model.load_state_dict(torch.load('gal_img1001/exp205_{}.pth'.format(num)))    
model.eval()


for data in test_dataloader:
    test_img_ = data
    test_img = test_img_.type(torch.float32)
    test_img = test_img.view(test_img.size(0), 1,64,64)
    test_img = test_img.cuda()
    test_img_output,_= model(test_img)
    test_latent = model.encoder(test_img)   #latent 
    

#x=(test_img.view(test_img.size(0),64,64)).data.cpu().numpy()
#x_hat=(test_img_output.view(test_img.size(0),64,64)).data.cpu().numpy()


In [ ]:
f = h5py.File('test.h5','r')
img = f['img'][:]
gal_flux = f['gal_flux'][:]
bulge_re = f['bulge_re'][:]
disk_n = f['disk_n'][:]
disk_r0 = f['disk_r0'][:]
bulge_frac= f['bulge_frac'][:]
gal_q = f['gal_q'][:]
gal_beta = f['gal_beta'][:]

f.close()


In [ ]:
b=img[0].max()
for i in range(3000): 
    if img[i].max() > b: 
        b=img[i].max()
    else: 
        b=b
        
    



In [ ]:
b/1e6